# SGD guarantees & the bias–variance trade-off

We will try to answer two:
1. Can we understand what the training process (SGD) does in a simplified setting? 
2. How to pick a model considering its expected error?

## SGD convergence guarantees

````{prf:definition}
:label: convexity_def
We call a function $f:\mathbb{R}^n \to \mathbb{R}$ convex if for all $t\in[0,1]$

$$f(tx + (1-t)y) \le t f(x) + (1-t)f(y) \ \forall x, y.$$
````

````{prf:lemma}
:label: convex_lemma

If $f:\mathbb{R}^n\to\mathbb{R}$ is differnetiable, it is convex iff

$$f(x) - f(y) \le \langle \nabla f(x), x-y\rangle \forall x, y.$$
````

````{prf:lemma} A bound on functions with Lipschitz gradient
:label: blg_lemma

If $f:\mathbb{R}^n\to\mathbb{R}$ has an L-Lipschitz gradient, we have

$$f(y) \le f(x) + \langle \nabla f(x), y-x\rangle + \frac{L}{2}\|y-x\|_2^2 \; \forall x, y.$$
````

```{admonition} Click for proof.
:class: dropdown
````{prf:proof}
Sketch:

1. Write out the first/second order Taylor expansion.
2. Use the Taylor inequality.
3. Bound the Taylor inequality using $L$.

$\square$
```

````{prf:theorem} Convergence of SGD
:label: sgd_thm

We assume that $f:\mathbb{R}^n\to\mathbb{R}$ has an L-Lipschitz gradient, and is convex. We also assume that we use stochastic gradients $v_k$ from an unbiased estimator with a bounded variance

$$\mbox{Var}(v_k) = \mathbb{E}\|v_k\|_2^2 - \|\mathbb{E}v_k\|_2^2 \le \sigma^2.$$

Then, for any SGD iteration $k$ with step size $t_k=t\le \frac{1}{L}$ we have

$$\mathbb{E} f(\overline x_k) \le f(x^*) + \frac{\|x_0-x^*\|_2^2}{2tk} + t\sigma^2,$$

where $\overline x_k = \frac{1}{k} \sum_{i=1}^{k}x_{i}$ and $x^* = \arg\min_x f(x)$.

Further, we can *expect* to find an $\epsilon/2$ optimal value (in terms of its loss) within

$$k = \frac{(2\sigma^2 + \|x_0-x^*\|_2^2)^2}{\epsilon^2}$$

iterations, by setting $t = \frac{1}{\sqrt{k}}$. Here we assume $\frac{1}{\sqrt{k}} \le \frac{1}{L}$.
````

```{admonition} Click for proof.
:class: dropdown
````{prf:proof}
The lemma above gives us

$$f(x_{i+1}) \le f(x_i) + \langle\nabla f(x_i), x_{i+1} - x_i\rangle + \frac{L}{2}\|x_{i+1} - x_i\|_2^2$$

$$= f(x_i) - t\langle\nabla f(x_i),v_i\rangle + \frac{Lt^2}{2}\|v_i\|_2^2.$$

Taking the expectation and using the bound on $t$ gives us

$$\mathbb{E}f(x_{i+1}) \le f(x_i) - t \|\nabla f(x_i)\|_2^2 + \frac{Lt^2}{2} [ \|\nabla f(x_i)\|_2^2 + \mbox{Var}(v_i)]$$

$$\le f(x_i) - t (1-\frac{Lt}{2})\|\nabla f(x_i)\|_2^2 + \frac{Lt^2}{2}\sigma^2$$

$$\le f(x_i) - \frac{t}{2} \|\nabla f(x_i)\|_2^2 + \frac{t}{2}\sigma^2.$$

Via convexity we know

$$f(x^*) + \langle\nabla f(x_i), x_i - x^*\rangle \ge f(x_i)$$

and therefore we have

$$\mathbb{E}f(x_{i+1}) \le f(x^*) + \langle\nabla f(x_i), x_i - x^*\rangle - \frac{t}{2} \|\nabla f(x_i)\|_2^2 + \frac{t}{2}\sigma^2.$$

Since we assume an unbiased estimator this can be written as

$$\mathbb{E}f(x_{i+1}) \le f(x^*) + \langle\mathbb{E} v_i, x_i - x^*\rangle - \frac{t}{2} \|\mathbb{E} v_i\|_2^2 + \frac{t}{2}\sigma^2$$

$$\mathbb{E}f(x_{i+1}) \le f(x^*) + \langle\mathbb{E} v_i, x_i - x^*\rangle - \frac{t}{2} (\mathbb{E}\|v_k\|_2^2 - \text{Var}(v_k)) + \frac{t}{2}\sigma^2$$

$$= f(x^*) + \mathbb{E} \left[\langle v_i, x_i - x^*\rangle - \frac{t}{2} \| v_i\|_2^2\right] + t\sigma^2$$

$$= f(x^*) + \mathbb{E} \left[\frac{1}{2t}\left(\|x_i-x^*\|_2^2 - \|x_i-x^*-tv_i\|_2^2\right)\right] + t\sigma^2$$

$$= f(x^*) + \mathbb{E} \left[\frac{1}{2t}\left(\|x_i-x^*\|_2^2 - \|x_{i+1}-x^*\|_2^2\right)\right] + t\sigma^2.$$

By taking a sum and noticing the canceling terms we have

$$\sum_{i=0}^{k-1} \mathbb{E} f(x_{i+1}) - f(x^*) \le \frac{1}{2t} \left(\|x_0 - x^*\|_2^2 - \mathbb{E}\|x_k-x^*\|_2^2\right) + kt\sigma^2$$

$$\le \frac{\|x_0-x^*\|_2^2}{2t} + kt\sigma^2$$

and thereby

$$\mathbb{E}\frac{1}{k}\sum_{i=0}^{k-1} f(x_{i+1}) \le f(x^*) + \frac{\|x_0-x^*\|_2^2}{2tk} + t\sigma^2.$$

Jensen's inequality, i.e., convexity, now gives us

$$\mathbb{E} f(\overline x_k) = \mathbb{E} f(\frac{1}{k}\sum_{i=0}^{k-1}x_{i+1}) \le f(x^*) + \frac{\|x_0-x^*\|_2^2}{2tk} + t\sigma^2.$$


We will leave the "number of iterations" part of the result as an exercise.

$\square$
```

````{note}
"Hard" convergence results of SGD usually require a non-constant step size, e.g., $\mbox{step size} = \frac{1}{\mbox{current iteration}}$.
````

## The bias–variance trade-off

We assume a regression dataset

$$\mathscr{D} =\{(x_i,y_i)\}_{i=1}^N \subset \mathbb{R}^n \times \mathbb{R}$$

of realizations $(x_i, y_i)\sim p_{X,Y}$.

And we assume that the labels, $y$, are not deterministicly determined by the features $x$.
As this means that we can have mutliple labels, it makes sense to think about the expected label for a given feature vector $x$.

* The **expected label** is defined as

$$\overline y(x) = \mathbb{E}_{y|x} Y = \int_\mathbb{R} y\ d\mathbb{P}(y|x) = \int_\mathbb{R} y\ p(y|x) dy.$$

* We will now also define the trained model based on some deterministic (in practice you could use a fixed seed) approach $\mathcal{A}$ and the dataset as

$$f_\mathcal{D} = \mathcal{A}(\mathcal{D}).$$


* As $\mathcal{D}$ is randomly drawen from $p_{X,Y}$ we can also think about the **expected model** we get from an approach $\mathcal{A}$. The approach describes the whole process of producing the model. This includes the network architecture, initialization, optimizer, learning rate, ... We define the **expected model** as

$$\overline f = \mathbb{E}_\mathcal{D} f_\mathcal{D} = \int_{\mathbb{R}^{(n+1) \times N}} f_\mathcal{D} \ d\mathbb{P}(\mathcal{D}).$$



* Having these concepts in place allows us to think about the **expected test error**

$$\mathbb{E}_{(x,y)\sim p_{x,y}, \mathcal{D}\sim p_{X,Y}^N} \left(f_\mathcal{D}(x) - y\right)^2.$$

````{prf:theorem} Bias-variance decomposition
:label: bv_thm
We can decompose the expected test error in the following way:

$$\mathbb{E}_{\substack{(x,y)\sim p_{x,y}\\ \mathcal{D}\sim p_{X,Y}^N}} \left(f_\mathcal{D}(x) - y\right)^2 = \mathbb{E}_{x, \mathcal{D}} (f_\mathcal{D}(x) - \overline f(x))^2 + \mathbb{E}_x (\overline f(x) - \overline y(x))^2 + \mathbb{E}_{x,y} (\overline y(x) - y)^2.$$

We can interpret these terms of the sum as follows:
* $\mathbb{E}_{x, D} (f_\mathcal{D}(x) - \overline f(x))^2$
    + The **variance** our approach has for different datasets of size $N$ from $p_{X,Y}$.
    + I.e., if we would have had a different dataset, how different would we expect our model to be.
* $\mathbb{E}_x (\overline f(x) - \overline y(x))^2$
    + The **bias** our approach has for different datasets of size $N$ from $p_{X,Y}$.
    + I.e., does our approach produce models with a systematic error, e.g., values that are always too large.
* $\mathbb{E}_{x,y} (\overline y(x) - y)^2$
     + This is the **noise** inherent to the label due to its non-deterministic nature w.r.t. the features.
    + I.e., the stuff that is impossible to know.
````

```{admonition} Click for proof.
:class: dropdown
````{prf:proof}
We have

$$
\mathbb{E}_{x,y,D}\left[f_{D}(x) - y\right]^{2}
= \mathbb{E}_{x,y,D}\left[\left(f_{D}(x) - \overline f(x)\right) + \left(\overline f(x) - y\right)\right]^{2}\nonumber$$
$$
= \mathbb{E}_{x, D}(\overline f_{D}(x) - \overline f(x))^{2} + \mathbb{E}_{x, y} \left(\overline f(x) - y\right)^{2} + 2 \mathbb{E}_{x, y, D} \left(f_{D}(x) - \overline f(x)\right)\left(\overline f(x) - y\right)
$$

By splitting up $\mathbb{E}_{x, y, D} = \mathbb{E}_{x, y}\mathbb{E}_{D}$ we see that the last term vanishes.

Now we only need to show that the second term decomposes into the desired outcome.
By expanding we find

$$\mathbb{E}_{x,y} (\overline f(x) - y)^2 = \mathbb{E}_{x,y} \left[(\overline f(x) - \overline y(x)) + (\overline y(x) - y)\right]^2$$
$$= \mathbb{E}_{x,y} (\overline f(x) - \overline y(x))^2 + \mathbb{E}_{x,y} (\overline y(x) - y)^2 + 2\mathbb{E}_{x,y} [\overline f(x) -\overline y(x)][\overline y(x) - y].$$

Again, by decomposing an expectation, this time $\mathbb{E}_{x,y} = \mathbb{E}_{x}\mathbb{E}_{y|x}$, we see that the last term vanishes. $\square$
```

Having a high variance and low bias tends to lead to overfitting, while having a low variance and high bias tends to lead to underfitting.

```{figure} images/bullseye.png
---
height: 400px
---
Bias vs variance. [Source.](http://scott.fortmann-roe.com/docs/BiasVariance.html)
```

```{figure} images/biasvariance.png
---
height: 350px
---
The bias-variance trade-off. On the left, the model underfits; on the right, it overfits. [Source.](http://scott.fortmann-roe.com/docs/BiasVariance.html)
```

```{figure} images/over_under_fitting.png
---
height: 200px
---
Over-, and under fitting for a one-dimensional regression problem. [Source.](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)
```

```{figure} images/over_under_fitting_classification.jpg
---
height: 200px
---
Over-, and under fitting for a two-dimensional classification problem. [Source.](https://datahacker.rs/018-pytorch-popular-techniques-to-prevent-the-overfitting-in-a-neural-networks/)
```

````{note}
Recently people started to investigate new thinking frameworks about generalization and the bias-variance trade-off. These efforts are motivated by currently not understood observed phenomena in Deep Learning.
````

## Exercises

### SGD
* A common question in DL is: If I increase my step size by a factor of $\alpha$, how should I change my batch size to get a similar behavor in my loss? Sometimes the answer is: You have to keep the variance of your stochastic gradient update constant to get the same result. Thanks to the limit theorem this means you get similar behavor if you keep $\frac{\text{step size}}{\sqrt{\text{batch size}}}$ constant. If we use the theorem about the convergence of SGD and the central limit theorem, and try to answer the question in the limit in terms of the ball we converge into, we get the answer to keep $\frac{\text{step size}}{\text{batch size}}$ constant. Derive both fractions.
```{toggle}
The first fraction results from looking at the variance of the stochastic gradient update:

$$\text{var}(t\frac{1}{\text{batch size}}\sum_i g_i) = \frac{t^2}{n}\sum_i\text{var}(g_i),$$

here $g_i$ are stochastic gradients.

The second fraction we get if we look at

$$\mathbb{E} f(\overline x_k) \le f(x^*) + \frac{\|x_0-x^*\|_2^2}{2tk} + \frac{t\sigma^2}{2}$$

and consider that central limit theorem tells us that $\sigma^2\propto \frac{1}{\text{batch size}}$.

 This means that if we want to converge into a ball given by $\frac{t\sigma^2}{2}$ of the same size, we have the scaling law that the size of the ball does not change if $\frac{\text{step size}}{\text{batch size}}$ stays constant. This obviously only holds for $t L \le 1$.

```
* What advantage does SGD have over GD from a compute/memory point of view?
* What advantage does SGD w.r.t. to instable stationary points.
* Train simple MLPs on MNIST.
    1. One for each of the following step size and batch size pairs: $(1e-4, 2)$, $(5e-4, 10)$, $(1e-3, 20)$, $(5e-4, 50)$, $(1e-3, 200)$. Keep all other parameters constant.
    2. Create of contraining the training loss curves of all the trained models.
    3. Analyzte the two losses curve sets given by the relationships $\frac{\mbox{step size}}{\mbox{batch size}}=\mbox{const}$ and $\frac{\mbox{step size}}{\sqrt{\mbox{batch size}}}=\mbox{const}$.
* Show the "number of iterations" part of the theorem.

```{toggle}
We want a bound on $\epsilon/2$. We begin with

$$\mathbb{E} f(\overline x_k) - f(x^*) \le \epsilon/2 \le \frac{\|x_0-x^*\|_2^2}{2tk} + t\sigma^2.$$

This gives

$$t\epsilon \le \frac{\|x_0-x^*\|_2^2}{k} + 2t^2\sigma^2$$

and by plugging in $t=1/\sqrt{k}$ we get

$$\frac{1}{\sqrt{k}}\epsilon \le \frac{\|x_0-x^*\|_2^2}{k} + \frac{2}{k}\sigma^2$$

$$\sqrt{k} \le \frac{\|x_0-x^*\|_2^2 + 2\sigma^2}{\epsilon}.$$

Squaring both sides completes the proof.
```

### Bias-variance
* Name two things that can cause overfitting.
```{toggle}
1. Too little trainig data.
2. A too large/complex model.
```
* Complete the missing parts of the Bias-variance decomposition proof.
* Would a linear regression under- or overfit on MNIST?
* How can the split into a training, validation, and test set help to avoid overfitting?
* Train an MLP on MNIST that overfits.
* Train an MLP on MNIST that underfits.
* When fitting a general polynomial of degree $p$ to data, how does the degree of the polynomial influence over- and underfitting?